In [195]:
import requests
import json
import pandas as pd 
from datetime import datetime
from importlib import reload 
import gzip 
import connection


connection = reload(connection) 


<connection object at 0x784c27142ac0; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>
Connection to Postgres Server: coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com is successful on Port: 5439. Database is dev
Connection to Postgres Server: 172.17.0.2 is successful on Port: 5432. Database is exchange_rate


In [136]:
data = {
    'disclaimer': "https://openexchangerates.org/terms/",
    'license': "https://openexchangerates.org/license/",
    'timestamp': 1449877801,
    'base': "USD",
    'rates': {
        'AED': 3.672538,
        'AFN': 66.809999,
        'ALL': 125.716501,
        'AMD': 484.902502,
        'ANG': 1.788575,
        'AOA': 135.295998,
        'ARS': 9.750101,
        'AUD': 1.390866
       
    }
}

In [138]:
# data = json.loads(data)
# data
df = pd.DataFrame(data)
df

,disclaimer,license,timestamp,base,rates
AED,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,3.672538
AFN,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,66.809999
ALL,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,125.716501
AMD,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,484.902502
ANG,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,1.788575
AOA,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,135.295998
ARS,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,9.750101
AUD,https://openexchangerates.org/terms/,https://openexchangerates.org/license/,1449877801,USD,1.390866


In [2]:
# create connection object postgres db 
local_con =connection.local_pgcon
local_cur = connection.local_pgcur

print(local_con) 
print("\n")
print(local_cur) 


<connection object at 0x784c3cac2ac0; dsn: 'user=postgres password=xxx dbname=coinbase10 host=172.17.0.2 port=5432', closed: 0>


<built-in method cursor of psycopg2.extensions.connection object at 0x784c3cac2ac0>


In [179]:
# create s3 object
s3 = connection.s3_bucket

# create Redshift connection and cursor
cur_redshift = connection.redshift_cur
conn_redshift = connection.redshift_con
# print(cur_redshift)
print(cur_redshift)

#create local postgres connection and cursor
cur_pg = connection.local_pgcur
conn_pg = connection.local_pgcon



<built-in method cursor of psycopg2.extensions.connection object at 0x784c33500f40>


In [4]:
# delete this block
# conn_redshift = connection.conn_redshift
# cur_redshift = connection.cur_redshift
# print(cur_redshift)

In [60]:
# open exchange
url2 ='https://api.binance.com/api/v3/ticker/price'

url = 'https://api.coinbase.com/v2/currencies'
# connection.api_url 
response = requests.get(connection.api_url)         

In [168]:
api_data = response.text
api_data

'{\n  "disclaimer": "Usage subject to terms: https://openexchangerates.org/terms",\n  "license": "https://openexchangerates.org/license",\n  "timestamp": 1716120007,\n  "base": "USD",\n  "rates": {\n    "AED": 3.673,\n    "AFN": 71.479051,\n    "ALL": 92.148131,\n    "AMD": 386.030742,\n    "ANG": 1.792758,\n    "AOA": 849,\n    "ARS": 881.603415,\n    "AUD": 1.492315,\n    "AWG": 1.8025,\n    "AZN": 1.7,\n    "BAM": 1.795053,\n    "BBD": 2,\n    "BDT": 116.488448,\n    "BGN": 1.79898,\n    "BHD": 0.375109,\n    "BIF": 2853.833157,\n    "BMD": 1,\n    "BND": 1.340645,\n    "BOB": 6.87348,\n    "BRL": 5.1046,\n    "BSD": 1,\n    "BTC": 0.000014849738,\n    "BTN": 82.888175,\n    "BWP": 13.478959,\n    "BYN": 3.254922,\n    "BZD": 2.00514,\n    "CAD": 1.36175,\n    "CDF": 2775.346609,\n    "CHF": 0.911919,\n    "CLF": 0.032417,\n    "CLP": 894.49773,\n    "CNH": 7.2317,\n    "CNY": 7.2232,\n    "COP": 3807.1681,\n    "CRC": 509.113839,\n    "CUC": 1,\n    "CUP": 25.75,\n    "CVE": 101.20

In [169]:
api_data = f''' {api_data} '''
api_data
api_data = json.loads(api_data)
# api_data
df = pd.DataFrame(api_data)

# extract base and timestamp column data
base = api_data['base']
timestamp = api_data['timestamp']

# Get date and time from timestamp 
date = datetime.fromtimestamp(timestamp)
# format date to YYYYMMDD to be used as a column in the dataframe
year = date.strftime('%Y%m%d')
# format time to HMS
time = date.strftime('%H%M%S')
# this will be part of the csv filename to ensure uniqueness of filename 
rate_date = year + '_'+ time

df = pd.DataFrame(api_data['rates'].items(), columns=['Currency', 'Rate'])

# Add base, timestamp, and date columns
df['Base'] = base
df['Timestamp'] = timestamp 
df['Date'] = year



In [212]:
rate_date = year + '_'+ time

In [166]:
api_data

{'disclaimer': 'Usage subject to terms: https://openexchangerates.org/terms',
 'license': 'https://openexchangerates.org/license',
 'timestamp': 1716120007,
 'base': 'USD',
 'rates': {'AED': 3.673,
  'AFN': 71.479051,
  'ALL': 92.148131,
  'AMD': 386.030742,
  'ANG': 1.792758,
  'AOA': 849,
  'ARS': 881.603415,
  'AUD': 1.492315,
  'AWG': 1.8025,
  'AZN': 1.7,
  'BAM': 1.795053,
  'BBD': 2,
  'BDT': 116.488448,
  'BGN': 1.79898,
  'BHD': 0.375109,
  'BIF': 2853.833157,
  'BMD': 1,
  'BND': 1.340645,
  'BOB': 6.87348,
  'BRL': 5.1046,
  'BSD': 1,
  'BTC': 1.4849738e-05,
  'BTN': 82.888175,
  'BWP': 13.478959,
  'BYN': 3.254922,
  'BZD': 2.00514,
  'CAD': 1.36175,
  'CDF': 2775.346609,
  'CHF': 0.911919,
  'CLF': 0.032417,
  'CLP': 894.49773,
  'CNH': 7.2317,
  'CNY': 7.2232,
  'COP': 3807.1681,
  'CRC': 509.113839,
  'CUC': 1,
  'CUP': 25.75,
  'CVE': 101.202307,
  'CZK': 22.718,
  'DJF': 177.137312,
  'DKK': 6.8646,
  'DOP': 58.271764,
  'DZD': 134.641,
  'EGP': 46.817497,
  'ERN': 15,

In [213]:
csv_file_path = f'exchange_rate_{rate_date}.csv' 
# date = datetime.utcfromtimestamp(timestamp)
# date
df.to_csv(csv_file_path, index=False)

In [49]:
# Check if the request was successful (status code 200)
if response.status_code == 200:

    api_data = response.text
   
    api_data = json.loads(api_data)
        # use this code if the result of the parsed json is in the form {data: {all the response data}}
    # api_data = api_data.get('data', [])

    # Read the JSON data into a Pandas DataFrame
    df = pd.DataFrame(api_data)
    
    # Specify the file path where you want to save the CSV file
    csv_file_path = 'saturday_rate.csv'
    
    # Write the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    
    print("CSV file saved successfully:", csv_file_path)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

CSV file saved successfully: saturday_rate.csv


In [214]:
# creating a filename variable for compressed file to be uploaded to S3
file_name = csv_file_path + '.gz'
print(file_name)

exchange_rate_20240519_120007.csv.gz


In [153]:
df.to_csv(file_name, index=False, compression='gzip')

In [215]:
# df to zip

df.to_csv(file_name, index=False, compression='gzip')

compressed_csv = file_name


In [216]:
with open(compressed_csv, 'rb') as file:
    # Upload the file object to S3 bucket
    s3.upload_fileobj(file, 'emil-coinbase-bucket', compressed_csv)

In [217]:
# Check if the file has been successfully uploaded
if s3.head_object(Bucket='emil-coinbase-bucket', Key=compressed_csv):
    print("CSV file uploaded to S3 bucket successfully")
else:
    print("Upload not successful")

CSV file uploaded to S3 bucket successfully


In [197]:
try:
    table_name = 'exchange_rates'
    rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        currency VARCHAR(3),
                        rate NUMERIC(16, 8),
                        base VARCHAR(3),
                        timestamp VARCHAR(16),
                        date VARCHAR(8)
                    )'''
    connection.cur_pg.execute(rates_table)
    connection.conn_pg.commit()
except conn_pg.Error as e:
    connection.conn_pg.rollback()
    print("Error occurred:", e)



In [196]:
# INSERT DATA TO LOCAL DB
try:
    # size of each batch
    batch_size = 1000

    # Insert DataFrame into database in batches
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]
        values = [tuple(row) for row in batch_df.values]

        placeholders = ','.join(['%s'] * len(df.columns))
        try:
            insert_query = f"INSERT INTO {table_name} (currency, rate, base, timestamp, date) VALUES ({placeholders})"
            connection.cur_pg.executemany(insert_query, values)
            connection.conn_pg.commit()

            print(insert_query)
        except Exception as e:
            # Rollback the transaction
            connection.conn_pg.rollback()

            # If the insert fails due to a unique constraint violation, perform an update instead
            if 'duplicate key' in str(e):
                for value in values:
                    update_query = "UPDATE currency_rates SET Name = %s, min_size = %s WHERE ID = %s"
                    connection.cur_pg.execute(update_query, (value[1], value[2], value[0]))
                    connection.conn_pg.commit()  # Commit each update
            else:
                raise e

except Exception as e:
    print("An error occurred during batch data insertion:", e)
    connection.conn_pg.rollback()  # Rollback the transaction in case of an error



INSERT INTO exchange_rates (currency, rate, base, timestamp, date) VALUES (%s,%s,%s,%s,%s)


In [206]:
# upsert on conflict
# INSERT DATA TO LOCAL DB
try:
    # size of each batch
    batch_size = 200

    # Insert DataFrame into database in batches
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]
        values = [tuple(row) for row in batch_df.values]

        placeholders = ','.join(['%s'] * len(df.columns))
        try:
            insert_query = f'''INSERT INTO {table_name} (currency, rate, base, timestamp, date) 
                   VALUES ({placeholders})'''
                   

            connection.cur_pg.executemany(insert_query, values)
            connection.conn_pg.commit()

            print(insert_query)
        except Exception as e:
            # Rollback the transaction
            connection.conn_pg.rollback()


except Exception as e:
    print("An error occurred during batch data insertion:", e)
    connection.conn_pg.rollback()  # Rollback the transaction in case of an error


INSERT INTO exchange_rates (currency, rate, base, timestamp, date) 
                   VALUES (%s,%s,%s,%s,%s)


In [139]:
# INSERT INTO users (id, rate)
# VALUES (1, 0)
# ON CONFLICT (id) DO UPDATE
# SET min_value = %s;


In [136]:
# Upload data to local postgres db 
    # create table
table_name = 'currency_rates'
rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (ID VARCHAR(3), Name VARCHAR(65), min_size numeric(10, 8) ) '''
cur_pg.execute(rates_table)
conn_pg.commit()
    # copy csv


# insert_from_local = f"""COPY currency_rates
# FROM '{csv_file_path}'
# DELIMITER ','
# CSV
# HEADER;
#  """

insert_from_local= f"""copy currency_rates FROM '{csv_file_path}' CSV HEADER DELIMITER ','; """


# insert_from_local = f"""COPY currency_rates
# FROM '{csv_file_path}'
# IGNOREHEADER 1
# FORMAT AS csv
# DELIMITER ','; """

cur_pg.execute(insert_from_local)
conn_pg.commit()

conn_pg.close()

UndefinedFile: could not open file "./scratch_csv.csv" for reading: No such file or directory
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.


In [122]:
#  delete this block
connection.cur_redshift.execute('ROLLBACK')

In [207]:
    # create table REDSHIFT

table_name = 'exchange_rates'
try:
    
 rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        currency VARCHAR(3),
                        rate NUMERIC(16, 8),
                        base VARCHAR(3),
                        timestamp VARCHAR(16),
                        date VARCHAR(8)
                    )'''
    
  
 connection.cur_redshift.execute(rates_table)
 connection.conn_redshift.commit()
 print("Table created successfully.")

except Exception as e:
 connection.conn_redshift.rollback()
 print(f"Error creating table: {e}")

Table created successfully.


In [223]:
# upload data to Redshift form s3 bucket
   

aws_access_key_id = connection.aws_access_key_id
aws_secret_access_key = connection.aws_secret_access_key

insert_from_s3 = f"""COPY {table_name}
FROM 's3://emil-coinbase-bucket/{compressed_csv}'
CREDENTIALS 'aws_access_key_id={aws_access_key_id};aws_secret_access_key={aws_secret_access_key}'
IGNOREHEADER 1
FORMAT AS csv
gzip
DELIMITER ','; """


connection.cur_redshift.execute(insert_from_s3)
connection.conn_redshift.commit()
print(f"Copy to Redshift: {table_name} was successful")

conn_redshift.close()




In [63]:
conn_redshift = connection.conn_redshift
cur_redshift = connection.cur_redshift
print(conn_redshift)

<connection object at 0x000002543DE7EA68; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>


In [221]:
connection.cur_redshift.execute('ROLLBACK')

In [42]:


)

None
